# Define variants

In [ ]:
mutations = [
    '12     121806070   T   C'.split(),
    '12     121808247   G   T'.split(),
    '12     121814593   C   G'.split(),
    '12     121817872   C   T'.split(),
    '12     121825271   C   T'.split(),
    '12     121827555   C   T'.split(),
    '12     121827739   G   C'.split(),
    '12     121827745   A   G'.split()
]

# Run SnpEff

In [ ]:
import sys, os
from pathlib import Path
import json

sys.path.append(os.fspath(Path.cwd() / 'mutation_classifier/programs'))

from SnpEff.SnpEff_ibex import SnpEffIbex

In [ ]:
out_dir = Path('./results/snpeff')

In [ ]:
exe = SnpEffIbex(all_variants_list, out_dir=out_dir, jobname='SnpEff', max_jobs=1000)
jobid = exe.run()

In [ ]:
!squeue -u guzmanfj

# Check results

In [ ]:
from pathlib import Path
import json

In [ ]:
!ls ./results/snpeff/ | wc

In [ ]:
pkls = list(Path('./results/snpeff').glob('*.pkl'))
len(pkls)

In [ ]:
dfs = []
for pkl in pkls:
    dfs.append(pd.read_pickle(pkl))

all_variants_snpeff = pd.concat(dfs)

# Identify LOF variants

In [ ]:
lof_effects = ['Chromosome_number_variation',
'Exon_loss_variant', 
'Frameshift_variant', 
'Stop_gained', 
'Stop_lost', 
'Start_lost', 
'Splice_acceptor_variant', 
'Splice_donor_variant', 
'Rare_amino_acid_variant',
'Transcript_ablation', 
'Disruptive_inframe_insertion', 
'Disruptive_inframe_deletion']

lof_effects = [e.lower() for e in lof_effects]

Reassign the `effect` column to separate multiple effects

In [ ]:
all_variants_snpeff['effect'] = all_variants_snpeff.effect.apply(lambda x: str(x).split('&'))

In [ ]:
all_variants_snpeff.effect

Create LOF classification

In [ ]:
all_variants_snpeff['LOF'] = all_variants_snpeff.effect.apply(lambda x: any([eff in lof_effects for eff in x]))

# Identify non synonymous variants

In [ ]:
nonsym_effects = ['Missense_variant' ,
'Inframe_insertion' ,
'Inframe_deletion' ,
'5_prime_UTR_truncation' ,
'3_prime_UTR_truncatisplice_region_variant' ,
'Splice_branch_variant' ,
'Coding_sequence_variant',
'Regulatory_region_ablation' ,
'TFBS_ablation',
'5_prime_UTR_premature_start_codon_gain_variant' ,
'Non-canonical_start_codon']

nonsym_effects = [e.lower() for e in nonsym_effects]

In [ ]:
all_variants_snpeff['non_syn'] = all_variants_snpeff.effect.apply(lambda x: any([eff in nonsym_effects for eff in x]))

# Output description
---

From https://pcingola.github.io/SnpEff/se_inputoutput/#ann-field-vcf-output-files

First 4 columns are the same as the input vcf. Columns 4 and 5 are repeated (bug). (?)

- **Allele (or ALT)**: In case of multiple ALT fields, this helps to identify which ALT we are referring to. E.g.:

- **Annotation (a.k.a. effect)**: Annotated using Sequence Ontology terms. Multiple effects can be concatenated using '&'.

- **Putative_impact**: A simple estimation of putative impact / deleteriousness : {HIGH, MODERATE, LOW, MODIFIER}

- **Gene Name**: Common gene name (HGNC). Optional: use closest gene when the variant is "intergenic".

- **Gene ID**: Gene ID

- **Feature type**: Which type of feature is in the next field (e.g. transcript, motif, miRNA, etc.). It is preferred to use Sequence Ontology (SO) terms, but 'custom' (user defined) are allowed.

- **Feature ID**: Depending on the annotation, this may be: Transcript ID (preferably using version number), Motif ID, miRNA, ChipSeq peak, Histone mark, etc. Note: Some features may not have ID (e.g. histone marks from custom Chip-Seq experiments may not have a unique ID).

- **Transcript biotype**: The bare minimum is at least a description on whether the transcript is {"Coding", "Noncoding"}. Whenever possible, use ENSEMBL biotypes.

- **Rank / total**: Exon or Intron rank / total number of exons or introns.

- **HGVS.c**: Variant using HGVS notation (DNA level)

- **HGVS.p**: If variant is coding, this field describes the variant using HGVS notation (Protein level). Since transcript ID is already mentioned in 'feature ID', it may be omitted here.

- **cDNA_position / cDNA_len**: Position in cDNA and trancript's cDNA length (one based).

- **CDS_position / CDS_len**: Position and number of coding bases (one based includes START and STOP codons).

- **Protein_position / Protein_len**: Position and number of AA (one based, including START, but not STOP).

- **Distance to feature**: All items in this field are options, so the field could be empty.

  - Up/Downstream: Distance to first / last codon
  - Intergenic: Distance to closest gene
  - Distance to closest Intron boundary in exon (+/- up/downstream). If same, use positive number.
  - Distance to closest exon boundary in Intron (+/- up/downstream)
  - Distance to first base in MOTIF
  - Distance to first base in miRNA
  - Distance to exon-intron boundary in splice_site or splice _region
  - ChipSeq peak: Distance to summit (or peak center)
  - Histone mark / Histone state: Distance to summit (or peak center)

- **Errors, Warnings or Information messages**: Add errors, warnings or informative message that can affect annotation accuracy. It can be added using either 'codes' (as shown in column 1, e.g. W1) or 'message types' (as shown in column 2, e.g. WARNING_REF_DOES_NOT_MATCH_GENOME). All these errors, warnings or information messages messages are optional.

Loss of function (LOF) and nonsense-mediated decay (NMD) annotations:

- **gene**: Gene name
- **geneid**: Gene ID (usually ENSEMBL)
- **ntranscripts**: Number of transcripts in this gene
- **fraction_affected**: Percentage of transcripts affected by this variant.